HW4_Sam Yuxiao Sun

In [2]:
#Import modules
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time
import datetime
import numpy as np

Part a & b

In [2]:
#Open the url with beautifual soup. 
base_url = "https://en.wikipedia.org"
index_ref = "/wiki/List_of_accidents_and_incidents_involving_commercial_aircraft"
index_html = urlopen(base_url + index_ref)
soup = BeautifulSoup(index_html, "lxml")

In [3]:
#Time delay function
def try_request(url):
    html = urlopen(url)
    time.sleep(1)
    return html

In [42]:
date=[] #set initial values
description=[]
link=[]
passengers=[]
crew=[]
fatalities=[]
survivors=[]
registration=[]
origin=[]
destination=[]
for each in soup.find_all('li'):
    if re.search('(–|-)(\s)(\w|\s|-|,|<i>)*<b><a', str(each)) is not None:  #match the '-'+link structure
        try:
            lin = each.find('a')['href'] #the link and description info can be directly scrapped from the main page
        except:
            lin=None
        try:
            de=each.get_text()
            desc=re.split(' –|- ',de)[1] #split with 'space'+'-' to aviod breaking up sentences
        except:
            try:
                desc=each.get_text() #in case of spliting failure
            except:
                desc=None
            
        html=try_request(base_url + lin)
        soup1= BeautifulSoup(html, "lxml")
        table=soup1.find('table',attrs={'class':'infobox'}) #extract the summary table from the sub-page
        
        try:
            da=table.find(text='Date').find_next('td').text.replace('\xa0','')#replace xa0 to decrease noise
            day=re.search('(?<!\d)\d{1,2}', da).group() #only one or two digits month
            month=re.search('([A-Z]|[a-z])+', da).group() #first word
            year=re.search('(\d\d\d\d)', da).group() #four digits year
            dat=' '.join([day,month,year]) 
        except:
            try:
                dat=table.find(text='Date').find_next('td').text #nested loop is created to support special senarios
            except:                             
                dat=None #I choose to use None instead of the string 'No Data' to make follow-on tasks easier
                
        try:
            p=table.find(text='Passengers').find_next('td').text
            pas=re.search('\d+', p).group() #select the first number, return the matched substring
        except:
            try:
                pas=table.find(text='Passengers').find_next('td').text
            except:
                pas=None
                
        try:
            c=table.find(text='Crew').find_next('td').text
            cre=re.search('\d+', c).group() 
        except:
            try:
                cre=table.find(text='Crew').find_next('td').text
            except:
                cre=None
                
#For fatalities and survivors, first try total number so that we can record the correct value for collision cases
        try:
            f=table.find(text='Total fatalities').find_next('td').text 
            fat=re.search('\d+', f).group()
        except:
            try:
                f=table.find(text='Fatalities').find_next('td').text
                fat=re.search('\d+', f).group()
            except:
                try:
                    fat=table.find(text='Fatalities').find_next('td').text
                except:
                    fat=None
                    
        try:
            s=table.find(text='Total survivors').find_next('td').text
            sur=re.search('\d+', s).group()
        except:
            try:
                s=table.find(text='Survivors').find_next('td').text
                sur=re.search('\d+', s).group()
            except:
                try:
                    sur=table.find(text='Survivors').find_next('td').text
                except:
                    sur=None
                    
        try:
            reg=table.find(text='Registration').find_next('td').text            
        except:
            reg=None
            
        try:
            ori=table.find(text='Flight origin').find_next('td').text
        except:
            ori=None
            
        try:
            dest=table.find(text='Destination').find_next('td').text
        except:
            dest=None
             
        description.append(desc) #append the list for each iteration
        link.append(lin)       
        date.append(dat)
        passengers.append(pas)
        crew.append(cre)
        fatalities.append(fat)
        survivors.append(sur)
        registration.append(reg)
        origin.append(ori)
        destination.append(dest)                      
    else:
        continue
df=pd.DataFrame({'description':description,'link':link,'date':date,'passengers':passengers,'crew':crew,'fatalities':fatalities,'survivors':survivors,'registration':registration,'origin':origin,'destination':destination})

In [134]:
df['date']=pd.to_datetime(df_air['date']) #use the datetime parser to convert str to datatime object
df

,crew,date,description,destination,fatalities,link,origin,passengers,registration,survivors
0,2,1919-08-02,"A Caproni Ca.48 crashes at Verona, Italy, dur...","Taliedo, Milan, Italy",14,/wiki/1919_Verona_Caproni_Ca.48_crash,"Venice, Italy",12,None,0
1,2,1922-04-07,"In the first mid-air collision of airliners, ...",Croydon,7,/wiki/First_mid-air_collision_of_airliners,"Le Bourget, Paris",3,F-GEAD,0
2,2,1923-05-14,An Air Union Farman F.60 Goliath crashes near...,"Croydon, Surrey, United Kingdom",6,/wiki/May_1923_Air_Union_Farman_Goliath_crash,"Le Bourget, Paris, France",4,F-AEBY,0
3,2,1923-08-27,An Air Union Farman F.60 Goliath crashes near...,"Croydon Airport, Surrey, United Kingdom",1,/wiki/August_1923_Air_Union_Farman_Goliath_crash,"Le Bourget Airport, Paris, France",11,F-AECB,12
4,1,1924-12-24,An Imperial Airways de Havilland DH.34 crashe...,"Le Bourget, Paris, France",8,/wiki/1924_Imperial_Airways_de_Havilland_DH.34...,"Croydon, Surrey, United Kingdom",7,G-EBBX,0
5,2,1926-08-18,An Air Union Blériot 155 crashes while attemp...,"Croydon Airport, Croydon, United Kingdom",3,/wiki/August_1926_Air_Union_Bl%C3%A9riot_155_c...,"Le Bourget Airport, Paris, France",13,F-AIEB,12
6,2,1926-10-02,An Air Union Blériot 155 catches fire in mid-...,"Croydon Airport, Croydon, United Kingdom",7,/wiki/October_1926_Air_Union_Bl%C3%A9riot_155_...,"Le Bourget Airport, Paris, France",5,F-AICQ,0
7,2,1927-08-22,"A KLM Fokker F. VIII crashes near Sevenoaks, ...","Waalhaven Airport, Rotterdam",1,/wiki/1927_KLM_Fokker_F.VIII_crash,Croydon Airport,9,H-NADU,10
8,1,1928-07-13,An Imperial Airways Vickers Vulcan crashes on...,"Croydon Airport, England",4,/wiki/1928_Imperial_Airways_Vickers_Vulcan_crash,"Croydon Airport, England",5,G-EBLB,2
9,2,1929-06-17,An Imperial Airways Handley Page W.10 ditches...,"Zurich Airport, Switzerland",7,/wiki/1929_Imperial_Airways_Handley_Page_W.10_...,"Croydon Airport, United Kingdom",11,G-EBMT,6


Part c

In [126]:
df2=df[['fatalities','origin','description']]

In [127]:
#It seems to be difficult to convert this column to int, so I use string comparision instead
df2.sort_values('fatalities',ascending=False).head(5) #top 5 fatalities

,fatalities,origin,description
456,583,"Los Angeles Int'l Airport\nLos Angeles, United...","in the Tenerife airport disaster, KLM Flight ..."
581,520,"Haneda Airport, Tokyo","Japan Airlines Flight 123, a Boeing 747, cras..."
758,349,"Indira Gandhi Int'l Airport\nDelhi, India",Charkhi Dadri mid-air collision: Saudi Arabia...
421,346,"Yesilköy Int'l Airport\nIstanbul, Turkey","Turkish Airlines Flight 981, a McDonnell Doug..."
578,329,Toronto (as Flight 181) Montréal-Mirabel Int'l...,"Air India Flight 182, a Boeing 747 en route f..."


Part d

In [128]:
df.columns=df.columns.str.strip() #strip off trailing and leading whitespace

In [139]:
df_25year=df[df['date']>'1991-02-24'] #last 25 years
#utilize the description column to count
df_25year.groupby('origin')['description'].agg('count').sort_values(ascending=False).head(1)

origin
Ninoy Aquino International Airport    3
Name: description, dtype: int64

Part e

In [137]:
import json
df.to_json('dataframe.json',date_format='iso') #specify date format so that "read_json" command can retrive datatime

In [8]:
df=pd.read_json('dataframe.json') #check the last row
df.loc[1057,:]

crew                                                            3
date                                          2016-02-24 00:00:00
description     Tara Air Flight 193, a de Havilland Canada DHC...
destination                                Jomsom Airport, Jomsom
fatalities                                                     23
link                                    /wiki/Tara_Air_Flight_193
origin                                   Pokhara Airport, Pokhara
passengers                                                     20
registration                                               9N-AHH
survivors                                                       0
Name: 1057.0, dtype: object

-----------------------------------------------------------------

Drafts

In [ ]:
#Example: scrape link
link = soup.find('li').find('a')['href']
link

In [6]:
#Table
html=urlopen(base_url + '/wiki/1942_Qantas_Short_Empire_shootdown')
soup1= BeautifulSoup(html, "lxml")
table=soup1.find('table',attrs={'class':'infobox'})
table

In [ ]:
#Date
try:
    da=table.find(text='Date').find_next('td').text
    day=re.search('(\d+)', da).group() #first value
    month=re.search('([A-Z]|[a-z])+', da).group() #first word
    year=re.search('(\d+){2}', da).group() #second value
    dat=' '.join([day,month,year]) 
except:
    try:
        dat=table.find(text='Date').find_next('td').text #this subloop is created to incoporate special senarios
    except:                             
        dat='No data'
dat

In [ ]:
#passengers
p=table.find(text='Passengers').find_next('td').text
passengers=re.search(r'\d+', p).group()
passengers

In [ ]:
#crews
c=table.find(text='Crew').find_next('td').text
crews=re.search(r'\d+', c).group()
crews

In [ ]:
#fatalities
f=table.find(text='Fatalities').find_next('td').text
fatalities=re.search(r'\d+', f).group()
fatalities

In [ ]:
#survivors
s=table.find(text='Survivors').find_next('td').text
survivors=re.search(r'\d+', s).group()
survivors

In [7]:
#registration
try:
    r=table.find(text='Registration').find_next('td').text
except:
    reg='No data'
reg

'No data'

In [ ]:
#origin
origin=table.find(text='Flight origin').find_next('td').text
origin

In [ ]:
#destination
destination=table.find(text='Destination').find_next('td').text
destination

In [ ]:
#Example: scrape description
d=soup.find('li').get_text()
des=re.split('–|-',d)[1]
des

In [ ]:
base1_url = "https://en.wikipedia.org"
index1_ref = "/wiki/May_1923_Air_Union_Farman_Goliath_crash"
index1_html = urlopen(base1_url + index1_ref)
soup2 = BeautifulSoup(index1_html, "lxml")

In [ ]:
da=soup2.find(text='Date').find_next('td').text.replace('\xa0','')
da
day=re.search('(?<!\d)\d{1,2}', da).group()
month=re.search('([A-Z]|[a-z])+', da).group()
year=re.search('(\d\d\d\d)', da).group()
dat=' '.join([day,month,year])
dat

In [16]:
df_25year=df[df['date']>'1901-02-24'] #last 25 years
#utilize the description column to count
df_25year.groupby('origin')['description'].agg('count').sort_values(ascending=False).head(1)

origin
London Heathrow Airport    10
Name: description, dtype: int64

In [18]:
df.sort_values('fatalities',ascending=False).head(5) #top 5 fatalities

,crew,date,description,destination,fatalities,link,origin,passengers,registration,survivors
456,16,1977-03-27,"in the Tenerife airport disaster, KLM Flight ...","Gran Canaria Airport\nCanary Islands, Spain",583,/wiki/Tenerife_airport_disaster,"Los Angeles Int'l Airport\nLos Angeles, United...",380,N736PA[1],61
581,15,1985-08-12,"Japan Airlines Flight 123, a Boeing 747, cras...","Osaka Int'l Airport, Itami",520,/wiki/Japan_Airlines_Flight_123,"Haneda Airport, Tokyo",509,JA8119,4
758,23,1996-11-12,Charkhi Dadri mid-air collision: Saudi Arabia...,"Dhahran International Airport\nDhahran, Saudi ...",349,/wiki/Charkhi_Dadri_mid-air_collision,"Indira Gandhi Int'l Airport\nDelhi, India",289,HZ-AIH,0
421,11,1974-03-03,"Turkish Airlines Flight 981, a McDonnell Doug...","London Heathrow Airport\nLondon, United Kingdom",346,/wiki/Turkish_Airlines_Flight_981,"Yesilköy Int'l Airport\nIstanbul, Turkey",335,TC-JAV,0
578,22,1985-06-23,"Air India Flight 182, a Boeing 747 en route f...","Indira Gandhi Int'l Airport\nNew Delhi, India",329,/wiki/Air_India_Flight_182,Toronto (as Flight 181) Montréal-Mirabel Int'l...,307,VT-EFO,0
